In [23]:
%%capture
! pip install pyarrow

In [35]:
import pandas as pd
import numpy as np

In [37]:
for col in df.columns:
    print(col)

birth_year
age
age_squared
female
white
black
hispanic
education_years
college_plus
self_rated_health
bmi
weight
height
mobility_limitations
large_muscle_limitations
adl_limitations
iadl_limitations
fine_motor_limitations
cognition_score
memory_recall
immediate_recall
delayed_recall
serial7
depression_score
felt_depressed
everything_effort
restless_sleep
felt_lonely
ever_smoked
current_smoker
drinks_per_day
drink_days_per_week
vigorous_activity
marital_status
self_rated_health_lag1
bmi_lag1
weight_lag1
height_lag1
mobility_limitations_lag1
large_muscle_limitations_lag1
adl_limitations_lag1
iadl_limitations_lag1
fine_motor_limitations_lag1
cognition_score_lag1
memory_recall_lag1
immediate_recall_lag1
delayed_recall_lag1
serial7_lag1
depression_score_lag1
felt_depressed_lag1
everything_effort_lag1
restless_sleep_lag1
felt_lonely_lag1
ever_smoked_lag1
current_smoker_lag1
drinks_per_day_lag1
drink_days_per_week_lag1
vigorous_activity_lag1
marital_status_lag1
self_rated_health_lag2
bmi_lag2

In [25]:
df = pd.read_parquet('data/processed/randhrs_diabetes_4yr.parquet')
df.head()

,birth_year,age,age_squared,female,white,black,hispanic,education_years,college_plus,self_rated_health,...,stopped_activity,drinks_per_week,heavy_drinking,age_x_bmi,sedentary_and_obese,depression_x_mobility,cognitive_decline_x_age,metabolic_risk_score,frailty_indicators,incident_diabetes_4yr
0,1936.0,60.0,3600.0,False,True,False,False,3.0,False,4.0,...,0,3.0,0,1596.0,0,0.0,-300.0,0,0,0.0
1,1938.0,58.0,3364.0,True,True,False,False,5.0,True,3.0,...,0,0.0,0,2024.2,0,0.0,-58.0,2,0,0.0
2,1939.0,57.0,3249.0,False,True,False,False,3.0,False,3.0,...,0,0.0,0,1350.9,0,0.0,57.0,0,0,0.0
3,1956.0,40.0,1600.0,True,True,False,False,5.0,True,3.0,...,0,0.0,0,1172.0,0,0.0,0.0,1,2,0.0
4,1939.0,57.0,3249.0,False,True,False,False,5.0,True,3.0,...,0,12.0,0,1692.9,0,0.0,114.0,0,0,0.0


In [26]:
print("Shape:", df.shape)
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

Shape: (98033, 134)
Number of rows: 98033
Number of columns: 134


In [27]:
target_col = [c for c in df.columns if c.startswith("incident_")][0]

n_features = df.drop(columns=[target_col]).shape[1]
print("Number of feature columns:", n_features)

Number of feature columns: 133


In [28]:
summary = df.describe().T[['mean', 'min', 'max', 'std']]
print(summary)

                                mean     min      max          std
birth_year               1940.546632  1896.0   1995.0    11.656099
age                        62.182734    15.0    101.0    10.615218
age_squared              3979.374170   225.0  10201.0  1348.765536
education_years             3.284694     1.0      5.0     1.365589
self_rated_health           2.655249     1.0      5.0     1.058333
...                              ...     ...      ...          ...
depression_x_mobility       2.030878     0.0     40.0     5.259431
cognitive_decline_x_age    22.323766 -1224.0   1620.0   232.559443
metabolic_risk_score        1.091439     0.0      4.0     1.027065
frailty_indicators          0.746851     0.0      3.0     0.800179
incident_diabetes_4yr       0.064846     0.0      1.0     0.246254

[129 rows x 4 columns]


In [29]:
df.select_dtypes(include='number').skew().sort_values(ascending=False).head(10)

drinks_per_week          7.287032
quit_smoking             6.698286
iadl_limitations_lag2    5.477801
iadl_limitations_lag1    5.045918
sedentary_and_obese      4.870582
heavy_drinking           4.833720
health_crash             4.770545
adl_limitations_lag2     4.684079
iadl_limitations         4.555633
adl_limitations_lag1     4.434964
dtype: float64

In [30]:
target_col = [c for c in df.columns if c.startswith("incident_")][0]
features = df.drop(columns=[target_col])

corr_matrix = features.corr(numeric_only=True)

corr_pairs = (
    corr_matrix.abs()
    .where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
) # pyright: ignore[reportCallIssue]

print(corr_pairs.head(20))

vigorous_activity      sedentary                  1.000000
                       physically_active          1.000000
ever_smoked            ever_smoked_lag1           0.999583
ever_smoked_lag1       ever_smoked_lag2           0.999572
ever_smoked            ever_smoked_lag2           0.999341
age                    age_squared                0.994318
cognition_decline_2yr  cognitive_decline_x_age    0.987415
height_lag1            height_lag2                0.984331
height                 height_lag1                0.980755
                       height_lag2                0.968896
weight_change_kg       weight_change_pct          0.964266
memory_recall          delayed_recall             0.949199
weight_lag1            weight_lag2                0.948989
memory_recall_lag1     delayed_recall_lag1        0.948761
memory_recall_lag2     delayed_recall_lag2        0.948373
weight                 weight_lag1                0.947596
bmi_velocity_2yr       weight_change_kg           0.9315

In [31]:
missing = df.isna().mean().sort_values(ascending=False)
print(missing.head(15))


vigorous_activity         0.737874
vigorous_activity_lag1    0.637642
vigorous_activity_lag2    0.582447
cognition_decline_4yr     0.258913
felt_lonely_lag2          0.244346
everything_effort_lag2    0.244336
felt_depressed_lag2       0.244081
restless_sleep_lag2       0.244071
depression_score_lag2     0.243918
cognition_score_lag2      0.243632
memory_recall_lag2        0.243632
immediate_recall_lag2     0.243632
delayed_recall_lag2       0.243632
serial7_lag2              0.243632
bmi_acceleration          0.232289
dtype: float64


In [32]:
constant_cols = [c for c in df.columns if df[c].nunique() <= 1]
print("Constant columns:", constant_cols)

Constant columns: []


In [33]:
[c for c in df.columns if "eligible_" in c or "target_" in c]

[]

In [34]:
print("Duplicate rows:", df.duplicated().sum())

Duplicate rows: 0
